## ETL Project
#### Extraction from 2 CSV files

In [1]:
#import required dependencies here:
import pandas as pd
import datetime
from sqlalchemy import create_engine

In [2]:
#read csv 1
url = "https://raw.githubusercontent.com/deritisn/analytics-portfolio/refs/heads/main/ETL_Projects/all_years_o3.csv"
df1 = pd.read_csv(url)
print(df1.head(5))

         Date Country          City Specie  count  min (ppb)  max (ppb)  \
0  2017-01-01      US   Los Angeles     o3     24        2.0       31.0   
1  2017-01-01      CN      Shanghai     o3    188        1.0       39.0   
2  2017-01-01      US     Manhattan     o3     24        3.0       28.0   
3  2017-01-01      US  Jacksonville     o3     46        2.0       18.0   
4  2017-01-01      CN       Beijing     o3    235        1.0        8.0   

   median (ppb)  
0          10.0  
1          11.0  
2          16.0  
3          12.0  
4           2.0  


In [3]:
df2 = pd.read_csv("https://raw.githubusercontent.com/deritisn/analytics-portfolio/refs/heads/main/ETL_Projects/all_years_pm25.csv")
df2.head()

,Date,Country,City,Specie,count,min (ug/m3),max (ug/m3),median (ug/m3)
0,2017-01-01,IN,New Delhi,pm25,24,54.9,282.7,177.7
1,2017-01-01,CN,Shanghai,pm25,215,12.0,63.2,34.4
2,2017-01-01,US,Jacksonville,pm25,115,3.1,113.6,8.1
3,2017-01-01,US,Los Angeles,pm25,69,3.3,109.7,15.9
4,2017-01-01,US,Manhattan,pm25,96,4.3,23.0,12.1


#### Transform

In [4]:
#drop last 3 columns
df1.drop(df1.columns[[5,6,7]], axis=1, inplace=True)
df1.head()

,Date,Country,City,Specie,count
0,2017-01-01,US,Los Angeles,o3,24
1,2017-01-01,CN,Shanghai,o3,188
2,2017-01-01,US,Manhattan,o3,24
3,2017-01-01,US,Jacksonville,o3,46
4,2017-01-01,CN,Beijing,o3,235


In [5]:
#rename the count columns
df1.columns = ['Date', 'Country', 'City','Specie', 'Count_o3']
df1.head()

,Date,Country,City,Specie,Count_o3
0,2017-01-01,US,Los Angeles,o3,24
1,2017-01-01,CN,Shanghai,o3,188
2,2017-01-01,US,Manhattan,o3,24
3,2017-01-01,US,Jacksonville,o3,46
4,2017-01-01,CN,Beijing,o3,235


In [6]:
#delete specie
df1.drop(df1.columns[[3]], axis=1, inplace=True)
df1.head()

,Date,Country,City,Count_o3
0,2017-01-01,US,Los Angeles,24
1,2017-01-01,CN,Shanghai,188
2,2017-01-01,US,Manhattan,24
3,2017-01-01,US,Jacksonville,46
4,2017-01-01,CN,Beijing,235


In [7]:
#drop last 3 columns
df2.drop(df2.columns[[5,6,7]], axis=1, inplace=True)
df2.head()

,Date,Country,City,Specie,count
0,2017-01-01,IN,New Delhi,pm25,24
1,2017-01-01,CN,Shanghai,pm25,215
2,2017-01-01,US,Jacksonville,pm25,115
3,2017-01-01,US,Los Angeles,pm25,69
4,2017-01-01,US,Manhattan,pm25,96


In [8]:
#rename the count columns
df2.columns = ['Date', 'Country', 'City','Specie', 'Count_pm25']
df2.head()

,Date,Country,City,Specie,Count_pm25
0,2017-01-01,IN,New Delhi,pm25,24
1,2017-01-01,CN,Shanghai,pm25,215
2,2017-01-01,US,Jacksonville,pm25,115
3,2017-01-01,US,Los Angeles,pm25,69
4,2017-01-01,US,Manhattan,pm25,96


In [9]:
#delete specie
df2.drop(df2.columns[[3]], axis=1, inplace=True)
df2.head()

,Date,Country,City,Count_pm25
0,2017-01-01,IN,New Delhi,24
1,2017-01-01,CN,Shanghai,215
2,2017-01-01,US,Jacksonville,115
3,2017-01-01,US,Los Angeles,69
4,2017-01-01,US,Manhattan,96


In [10]:
#merge the two dfs into one on date, country, city
#df = df1.merge(df2, on="Date", how="inner")
new_df = pd.merge(df1, df2,  how='left', left_on=['Date','Country', 'City'], right_on=['Date','Country', 'City'])
new_df.head()

,Date,Country,City,Count_o3,Count_pm25
0,2017-01-01,US,Los Angeles,24,69
1,2017-01-01,CN,Shanghai,188,215
2,2017-01-01,US,Manhattan,24,96
3,2017-01-01,US,Jacksonville,46,115
4,2017-01-01,CN,Beijing,235,386


#### Load

#### Schema Table

In [11]:
import os

In [12]:
os.listdir(os.getcwd())

['.ipynb_checkpoints',
 'Cities_and_Species_ETL.ipynb',
 'config_etl.py',
 'Instagram Reach Analysis using Python.ipynb',
 'Smartwatch Data Analysis using Python.ipynb']

In [13]:
import config_etl

In [14]:
print(config_etl.username)

postgres


In [15]:
pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
import psycopg2

In [27]:
#connect to local PostgreSQL DB
hostname = 'localhost'
database = 'ETL_Test'
username = config_etl.username
pwd = config_etl.password
port_id = 5432

conn = psycopg2.connect(
        host = hostname,
        dbname = database,
        user = username,
        password = pwd,
        port = port_id
    )

cursor = conn.cursor()
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cursor.fetchall())

[('merge_counts',)]


In [33]:
# Load my pandas dataframe
new_df.to_sql(name="merge_counts", con=engine, if_exists='append', index=False)

100

In [34]:
# Confirm data has been added to the table
pd.read_sql_query("SELECT * FROM merge_counts", con=engine).head()

,Date,Country,City,Count_o3,Count_pm25
0,2017-01-01,US,Los Angeles,24,69
1,2017-01-01,CN,Shanghai,188,215
2,2017-01-01,US,Manhattan,24,96
3,2017-01-01,US,Jacksonville,46,115
4,2017-01-01,CN,Beijing,235,386
